In [ ]:
# 📦 Imports
import networkx as nx
import matplotlib.pyplot as plt
import time
import os
import pandas as pd
import random
from collections import deque
import numpy as np
import matplotlib.animation as animation


In [2]:
def convert_edges_to_csv(edge_file):
    cleaned_data = []
    with open(edge_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith("#") or line.startswith("%") or line == "":
                continue
            parts = line.split()
            try:
                if len(parts) >= 3:
                    u, v, w = int(parts[0]), int(parts[1]), float(parts[2])
                elif len(parts) == 2:
                    u, v = int(parts[0]), int(parts[1])
                    w = 1.0
                else:
                    continue
                cleaned_data.append([u, v, w])
            except ValueError:
                continue

    df = pd.DataFrame(cleaned_data, columns=['source', 'target', 'weight'])
    csv_file = edge_file.replace('.edges', '.csv')
    df.to_csv(csv_file, index=False)
    print(f"✅ Saved cleaned CSV: {csv_file}")


In [3]:
def convert_amazon_format_to_csv(edge_file):
    cleaned_data = []
    user_map = {}
    item_map = {}
    user_counter = 0
    item_counter = 1_000_000  # Offset to keep users/items distinct

    with open(edge_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith("#") or line.startswith("%") or line == "":
                continue
            parts = line.split(",")
            if len(parts) < 3:
                continue
            user_id, item_id, rating = parts[0], parts[1], parts[2]

            try:
                rating = float(rating)
            except ValueError:
                continue

            # Map user to int
            if user_id not in user_map:
                user_map[user_id] = user_counter
                user_counter += 1
            u = user_map[user_id]

            # Map item to int
            if item_id not in item_map:
                item_map[item_id] = item_counter
                item_counter += 1
            v = item_map[item_id]

            cleaned_data.append([u, v, rating])

    df = pd.DataFrame(cleaned_data, columns=['source', 'target', 'weight'])
    csv_file = edge_file.replace('.edges', '.csv')
    df.to_csv(csv_file, index=False)
    print(f"✅ Saved cleaned CSV: {csv_file}")


In [8]:
edge_files = [
    "rec-amz-books.edges",
    "rec-amz-Video-Games.edges",
    "rec-movielens.edges",
    "rec-movielens-tag-movies-10m.edges",
    "rec-movielens-user-movies-10m.edges"
]



In [ ]:
for edge_file in edge_files:
    if "Video-Games" in edge_file or "books" in edge_file:
        convert_amazon_format_to_csv(edge_file)
    else:
        convert_edges_to_csv(edge_file)

In [4]:
def load_edge_list(file_path):
    df = pd.read_csv(file_path)
    G = nx.from_pandas_edgelist(df, 'source', 'target', edge_attr='weight')
    return G


In [5]:
def bfs_sample(G, target_nodes):
    visited = set()
    queue = deque()
    start_node = random.choice(list(G.nodes()))
    queue.append(start_node)
    visited.add(start_node)

    while len(visited) < target_nodes and queue:
        current = queue.popleft()
        for neighbor in G.neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
            if len(visited) >= target_nodes:
                break

    sampled_nodes = list(visited)
    return G.subgraph(sampled_nodes).copy()


In [6]:
def sample_connected_subgraph(G, target_nodes):
    if G.number_of_nodes() == 0:
        print("⚠️ Graph is empty. Skipping sampling.")
        return G

    if nx.is_connected(G):
        if G.number_of_nodes() <= target_nodes:
            return G
        else:
            return bfs_sample(G, target_nodes)

    # Graph is disconnected
    largest_cc = max(nx.connected_components(G), key=len)
    G_lcc = G.subgraph(largest_cc).copy()

    if G_lcc.number_of_nodes() < target_nodes:
        print(f"⚠️ Largest component only has {G_lcc.number_of_nodes()} nodes — using all of it.")
        return G_lcc

    return bfs_sample(G_lcc, target_nodes)


In [15]:
def fast_karger_mst(G, iterations=5):
    best_mst = []
    min_cost = float('inf')

    for _ in range(iterations):
        temp_G = G.copy()
        nodes = list(temp_G.nodes())
        mst_edges = []

        while len(temp_G.nodes()) > 1 and len(mst_edges) < len(G.nodes()) - 1:
            u, v = random.choice(list(temp_G.edges()))
            if temp_G.has_edge(u, v):
                weight = temp_G[u][v]['weight']
                mst_edges.append((u, v, weight))
                temp_G = nx.contracted_nodes(temp_G, u, v, self_loops=False)

        cost = sum(w for _, _, w in mst_edges)
        if cost < min_cost:
            min_cost = cost
            best_mst = mst_edges

    return best_mst


In [16]:
def karger_mst_with_steps(G, iterations=3):
    mst = fast_karger_mst(G, iterations)
    return mst, [mst]  # For animation compatibility



In [19]:
results = []
#smaller sample for karger
sample_size = {
    "rec-amz-books.csv": 800,
    "rec-amz-Video-Games.csv": 200,
    "rec-movielens.csv": 1000,
    "rec-movielens-tag-movies-10m.csv": 400,
    "rec-movielens-user-movies-10m.csv": 600,
}

dataset_files = [f.replace(".edges", ".csv") for f in edge_files]

for dataset in dataset_files:
    print(f"\n📂 Processing: {dataset}")
    G = load_edge_list(dataset)

    sample_target = sample_size.get(os.path.basename(dataset))

    if sample_target is not None:
        print(f"📉 Sampling {sample_target} connected nodes...")
        G = sample_connected_subgraph(G, sample_target)
        print(f"✅ Sampled nodes: {G.number_of_nodes()}, edges: {G.number_of_edges()}")


    start_time = time.time()
    mst_edges = fast_karger_mst(G, iterations=3)  # Start small
    end_time = time.time()



    total_cost = sum(w for _, _, w in mst_edges)
    exec_time = end_time - start_time


    print(f"✅ MST total cost: {total_cost:.2f}")
    print(f"⏱️ Execution time: {exec_time:.2f} seconds")

    results.append({
        "Dataset": os.path.basename(dataset),
        "Nodes": G.number_of_nodes(),
        "Edges": G.number_of_edges(),
        "MST Cost": total_cost,
        "Execution Time (s)": exec_time
    })



📂 Processing: rec-amz-books.csv
📉 Sampling 800 connected nodes...
✅ Sampled nodes: 800, edges: 875
✅ MST total cost: 3719.00
⏱️ Execution time: 4.87 seconds

📂 Processing: rec-amz-Video-Games.csv
📉 Sampling 200 connected nodes...
✅ Sampled nodes: 200, edges: 226
✅ MST total cost: 697.00
⏱️ Execution time: 0.30 seconds

📂 Processing: rec-movielens.csv
📉 Sampling 1000 connected nodes...
✅ Sampled nodes: 1000, edges: 26089
✅ MST total cost: 3477.50
⏱️ Execution time: 127.68 seconds

📂 Processing: rec-movielens-tag-movies-10m.csv
📉 Sampling 400 connected nodes...
✅ Sampled nodes: 400, edges: 1054
✅ MST total cost: 399.00
⏱️ Execution time: 1.98 seconds

📂 Processing: rec-movielens-user-movies-10m.csv
📉 Sampling 600 connected nodes...
✅ Sampled nodes: 600, edges: 1614
✅ MST total cost: 599.00
⏱️ Execution time: 4.44 seconds


In [20]:
df_results = pd.DataFrame(results)
df_results_sorted = df_results.sort_values(by='Nodes', ascending=True)
df_results_sorted.reset_index(drop=True, inplace=True)
df_results_sorted

,Dataset,Nodes,Edges,MST Cost,Execution Time (s)
0,rec-amz-Video-Games.csv,200,226,697.0,0.304600
1,rec-movielens-tag-movies-10m.csv,400,1054,399.0,1.984495
2,rec-movielens-user-movies-10m.csv,600,1614,599.0,4.442189
3,rec-amz-books.csv,800,875,3719.0,4.868739
4,rec-movielens.csv,1000,26089,3477.5,127.678461


In [24]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import networkx as nx

def animate_mst_steps(G, steps, save_path="animation.gif", max_frames=50):
    pos = nx.spring_layout(G, seed=42)  # fixed layout for consistent node positions
    fig, ax = plt.subplots(figsize=(6, 4))

    def update(i):
        ax.clear()
        ax.set_title(f"Step {i+1}")
        ax.axis('off')
        step_edges = steps[min(i, len(steps)-1)]
        nx.draw_networkx_nodes(G, pos, node_size=30, ax=ax)
        nx.draw_networkx_edges(G, pos, edgelist=step_edges, edge_color='red', width=2, ax=ax)

    frames = min(len(steps), max_frames)
    ani = animation.FuncAnimation(fig, update, frames=frames, interval=500, repeat=False)

    ani.save(save_path, writer='pillow', fps=2)
    plt.close(fig)
    print(f"🎥 GIF saved to: {save_path}")


In [25]:
dataset = "rec-amz-books.csv"
G = load_edge_list(dataset)
G = sample_connected_subgraph(G, 300)  # small sample for animation

mst_edges, steps = karger_mst_with_steps(G)  # or kruskal_mst_with_steps(G), etc.
total_cost = sum(w for _, _, w in mst_edges)
print(f"📊 Karger MST Cost: {total_cost:.2f}")

animate_mst_steps(G, steps, save_path="karger_animation.gif")


📊 Karger MST Cost: 1361.00
🎥 GIF saved to: karger_animation.gif
